In [1]:
# Training SVC with live audio data

from hearing_assistive import *
%matplotlib inline

# Get Silence Wave

In [2]:
empty_map = []
for i in range(6):
    empty = read_wave('Sounds/Output/Empty/empty0' + str(i) + '.wav')
    temp = empty.partition(seg_length=3)
    empty_map += temp
    
empty = read_wave('Sounds/Output/Silent_audio.wav')
temp = empty.partition(seg_length=3)
empty_map += temp

# Get Alarm Wave

In [3]:
alarm_map = []
for i in range(6):
    alarm = read_wave('Sounds/Output/LiveFireAlarm/live_fire_alarm0' + str(i) + '.wav')
    temp = alarm.partition(seg_length=3)
    alarm_map += temp
    
alarm = read_wave('Sounds/Output/FireAlarm_audio.wav')
temp = alarm.partition(seg_length=3)
alarm_map += temp

alarm = read_wave('Sounds/Output/bing_alarm.wav')
clip = alarm.segment(0, 7)
temp = clip.partition(seg_length=3)
alarm_map += temp

clip = alarm.segment(9, 11)
temp = clip.partition(seg_length=3)
alarm_map += temp

clip = alarm.segment(23, 10)
temp = clip.partition(seg_length=3)
alarm_map += temp

clip = alarm.segment(35, 12)
temp = clip.partition(seg_length=3)
alarm_map += temp

# Extract Features and Generate Training Set

In [4]:
import sys
import math

dataset = []
targetset = []

progress = 0.0
print "Extracting Alarm Features..."
for i in range(len(alarm_map)):
    new_dataset, new_targetset = generate_feature_set(alarm_map[i], 1)
    dataset, targetset = append_set(dataset, targetset, new_dataset, new_targetset)
    progress += (100.0 / len(alarm_map))
    sys.stdout.write('\r' + str(int(round(progress))) + ' %')   # show progress

progress = 0.0
print "\nExtracting Empty Features..."
for i in range(len(empty_map)):
    new_dataset, new_targetset = generate_feature_set(empty_map[i], 0)
    dataset, targetset = append_set(dataset, targetset, new_dataset, new_targetset)
    progress += (100.0 / len(empty_map))
    sys.stdout.write('\r' + str(int(round(progress))) + ' %')   # show progress
    
dataset = np.asarray(dataset)
targetset = np.asarray(targetset)

print '\n', dataset.shape

dataset = dataset.reshape((dataset.shape[0], -1))

Extracting Alarm Features...
100 %
Extracting Empty Features...
100 %
(2300, 149, 26)


# Train SVM Classifier

In [5]:
from sklearn import svm
from sklearn.externals import joblib

clf = svm.SVC(kernel='linear', C=100)
clf.fit(dataset, targetset)

joblib.dump(clf, 'SVC/svc_v2.pk1')

['SVC/svc_v2.pk1',
 'SVC/svc_v2.pk1_01.npy',
 'SVC/svc_v2.pk1_02.npy',
 'SVC/svc_v2.pk1_03.npy',
 'SVC/svc_v2.pk1_04.npy',
 'SVC/svc_v2.pk1_05.npy',
 'SVC/svc_v2.pk1_06.npy',
 'SVC/svc_v2.pk1_07.npy',
 'SVC/svc_v2.pk1_08.npy',
 'SVC/svc_v2.pk1_09.npy',
 'SVC/svc_v2.pk1_10.npy',
 'SVC/svc_v2.pk1_11.npy']

In [6]:
# NOTE:

# 40ms windows are better
# 26/26 mfccs are better
# kernel='linear' is better